# "Generative Adversial Network with Deep Fake for Image Animation"

**Repo cloning**

In [1]:
!git clone https://github.com/st7954/GAN_Deep_Fake_Project

Cloning into 'GAN_Deep_Fake_Project'...
remote: Enumerating objects: 65, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 65 (delta 1), reused 0 (delta 0), pack-reused 60
Receiving objects: 100% (65/65), 4.62 MiB | 2.43 MiB/s, done.
Resolving deltas: 100% (14/14), done.


In [2]:
cd GAN_Deep_Fake_Project

/content/GAN_Deep_Fake_Project


**Mount your Google drive folder on Colab**

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


**Updating the driving video and source image**

In [4]:
pip install imageio-ffmpeg

In [6]:
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
warnings.filterwarnings("ignore")

source_image = imageio.imread('/content/GAN_Deep_Fake_Project/Base_Source_Dataset/someone_face.png')
reader = imageio.get_reader('/content/GAN_Deep_Fake_Project/Base_Source_Dataset/US_president.mp4')


#Resize image and video to 256x256

source_image = resize(source_image, (256, 256))[..., :3]

fps = reader.get_meta_data()['fps']
driving_video = []
try:
    for im in reader:
        driving_video.append(im)
except RuntimeError:
    pass
reader.close()

driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani


HTML(display(source_image, driving_video).to_html5_video())

**Create a model and load checkpoints**

In [7]:
from keypointSourceCollector import load_checkpoints
generator, kp_detector = load_checkpoints(config_path='/content/gdrive/MyDrive/GAN/vox-256.yaml',
                            checkpoint_path='/content/gdrive/MyDrive/GAN/vox-adv-cpk.pth.tar')

Skipping
working


**Perform image animation**

In [8]:
from denseMotion import make_animation
from skimage import img_as_ubyte

predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True)

#save resulting video
imageio.mimsave('../generated.mp4', [img_as_ubyte(frame) for frame in predictions], fps=fps)
#video can be downloaded from /content folder

HTML(display(source_image, driving_video, predictions).to_html5_video())

100%|██████████| 211/211 [00:07<00:00, 27.50it/s]


In [9]:
predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=False, adapt_movement_scale=True)
HTML(display(source_image, driving_video, predictions).to_html5_video())

100%|██████████| 211/211 [00:07<00:00, 28.52it/s]


## Running on your data

**First we need to crop a face from both source image and video, while simple graphic editor like paint can be used for cropping from image. Cropping from video is more complicated. You can use ffpmeg for this.**

In [10]:
!ffmpeg -i /content/gdrive/My\ Drive/first-order-motion-model/07.mkv -ss 00:08:57.50 -t 00:00:08 -filter:v "crop=600:600:760:50" -async 1 hinton.mp4

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [11]:
source_image = imageio.imread('/content/GAN_Deep_Fake_Project/Base_Source_Dataset/someone_face.png')
driving_video = imageio.mimread('/content/GAN_Deep_Fake_Project/Base_Source_Dataset/Caprio.mp4', memtest=False)


#Resize image and video to 256x256

source_image = resize(source_image, (256, 256))[..., :3]
driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True,
                             adapt_movement_scale=True)

HTML(display(source_image, driving_video, predictions).to_html5_video())

100%|██████████| 444/444 [00:15<00:00, 28.98it/s]
